<a href="https://colab.research.google.com/github/revy1817/NLP_BG/blob/less_1_hw/HomeWork_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade nltk gensim bokeh pandas corus natasha annoy

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import pandas as pd
import numpy as np
import nltk
from tqdm import tqdm
import annoy

from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger

from corus import load_ods_gazeta
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, FastText
from sklearn.manifold import TSNE

from nltk.corpus import stopwords
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/Учеба/GeekBrains/Введение в обработку естественного языка/Less 3/gazeta.csv.gz'
records = load_ods_gazeta(path)

In [5]:
# Загрузим опредленное количество данных
data = []
for i in range(10000):
 data.append(next(records).text)

In [6]:
def preprocess_and_token_text(text):
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  return [token.lemma for token in doc.tokens if token.text not in russian_stopwords\
              and token.text != " " \
              and token.text.strip() not in punctuation\
              and len(token.text) != 1
          ]

In [7]:
# Параметры для предобработки текста
russian_stopwords = stopwords.words("russian")

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

vector_size = 1024

In [8]:
data_token = []

for text in tqdm(data):

  data_token.append(preprocess_and_token_text(text))

100%|██████████| 10000/10000 [26:43<00:00,  6.23it/s]


In [9]:
modelW2V = Word2Vec(sentences=data_token, vector_size=vector_size, window=4, min_count=1)

In [10]:
w2v_index = annoy.AnnoyIndex(vector_size ,'angular')

index_map = {}
counter = 0

for text in data:
  n_w2v = 0
  index_map[counter] = text
  vector_w2v = np.zeros(vector_size)

  for word in data_token[counter]:
    if word in modelW2V.wv:
      vector_w2v += modelW2V.wv[word]
      n_w2v += 1
  
  w2v_index.add_item(counter, (vector_w2v / n_w2v))
  counter += 1

w2v_index.build(10)

True

In [11]:
def get_response(question, index, model, index_map, vector_size):
    question = preprocess_and_token_text(question)
    vector = np.zeros(vector_size)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 3, )
    return [index_map[i] for i in answers]

In [15]:
get_response('война украина', w2v_index, modelW2V, index_map, vector_size)

['В Бундестаге раскритиковали и назвали прославлением нацизма прошедшее накануне в Киеве факельное шествие в честь экстремиста Степана Бандеры. Президент Чехии Милош Земан призвал чешское руководство выразить протест против «возвеличивания военных преступников» на Украине. Ранее президент Украины Петр Порошенко приравнял экстремистов ОУН-УПА, воевавших на стороне войск Третьего рейха, к украинским ветеранам Второй мировой войны, сражавшимся на стороне СССР.nДепутатnпарламента ФРГnот партии «Альтернатива для Германии» Петр Быстрон жестко раскритиковал прошедшее накануне в Киеве факельное шествие в честь националистаnСтепана Бандерыn(возглавлял Организацию украинских националистов — Украинскую повстанческую армию (организация запрещена в России), которая признана экстремистской организацией и чья деятельность запрещена на территории Российской Федерации).n«Думаю, это большая проблемаnукраинского правительстваn— прославление нацистского приспешника. Чудовищные преступления нацистов в ходе

In [16]:
get_response('Криптовалюта', w2v_index, modelW2V, index_map, vector_size)

['Глава Минэкологии России Сергей Донской подверг критике решение Трампа выйти из Парижского соглашения по климату, но отметил, что не удивлен, поскольку ранее Белый дом принял решение не ратифицировать Киотский протокол, который обязывает страны сократить или стабилизировать выбросы парниковых газов. Американский лидер ранее заявлял, что США будут продолжать тесно сотрудничать с участниками Парижского договора.nn#Adcenter_Vertical .partner_material_item{nmin-height: 45px;nmargin: 7px 20px;n}n#Adcenter_Vertical .sausage-header{nposition: relative;ndisplay: block;nheight: 37px;nmargin-left: 20px;nfont: bold 21px/38px \'PT Sans\', sans-serif;ncolor: #333;nletter-spacing: 0.01em;nborder-bottom: 1px solid #dfdfdf;n}n#Adcenter_Vertical .x80_80{nmax-width: 60px;nmax-height: 60px;nmargin-right: 10px;nfloat: left;n}n#Adcenter_Vertical .partner_material_item_text{nfloat:left;nmax-width: 175px;n}n#Adcenter_Vertical .partner_material_item_text_value{nfont-family: "PT Sans";nfont-size: 13px;nline-